### 개체명 인식: NER
- NER(Named Entity Recognition, 개체명 인식)에서는 문장 속에서 사람, 장소, 조직, 날짜 같은 의미 있는 단어들을 찾아내는 작업을 함.
- 텍스트에서 특정 의미를 가진 단어나 구절을 찾아내고 분류하는 작업
- 이때 단어(token)가 개체인지 표시하는 형식이 BIO 방식

In [1]:
# 홍길동은 2025년 11월 19일 서울시청에서 삼성전자 직원을 만났다
# 홍길동 - [인명]
# 2024년 1월 15일 - [날짜]
# 서울시청 - [지명]
# 삼성전자 - [기관명]

# 활용분야
    # 뉴스기사 : 기사에서 인물, 장소, 기관 자동 추출
    # 의료문서 : 병명, 약물명, 증상
    # 계약서 : 회사명, 날짜, 금액
    # 챗봇 : 사용자 질문에 핵심정보 파악

# BIO 태깅
    # B(Begin) 개체 시작
    # I(Inside) 개체 내부
    # O(Outside) 개체가 아님

In [60]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

In [61]:
# BIO 태깅
tokens = ["김철수는", "2024년", "1월", "15일", "서울시청에서", "삼성전자", "직원을", "만났다"]
bio_tags = ["B-PER", "B-DAT", "I-DAT", "I-DAT", "B-LOC", "B-ORG", "O", "O"]  #B-PER person's begin, B-DAT date's begin, B-LOC location's begin, ... # 직원을 만났다는 개체명이 아님. 
for token, tag in zip(tokens, bio_tags):
    if tag.startswith('B-'):
        desc = f"'{tag[2:]}' 개체의 시작"
    elif tag.startswith('I-'):
        desc = f"'{tag[2:]}' 개체의 내부"
    else :
        desc = "개체가 아님"
    print(f" {token:12} | {tag:8} | {desc}")

 김철수는         | B-PER    | 'PER' 개체의 시작
 2024년        | B-DAT    | 'DAT' 개체의 시작
 1월           | I-DAT    | 'DAT' 개체의 내부
 15일          | I-DAT    | 'DAT' 개체의 내부
 서울시청에서       | B-LOC    | 'LOC' 개체의 시작
 삼성전자         | B-ORG    | 'ORG' 개체의 시작
 직원을          | O        | 개체가 아님
 만났다          | O        | 개체가 아님


In [62]:
# 학습데이터
train_sentences = [
    ["김철수는", "서울에", "산다"],
    ["이영희는", "2024년에", "부산으로", "이사했다"],
    ["삼성전자는", "대한민국의", "대기업이다"],
    ["박지성은", "축구선수다"],
    ["2025년", "1월", "1일은", "새해다"],
]

train_labels = [
    ["B-PER", "B-LOC", "O"],
    ["B-PER", "B-DAT", "B-LOC", "O"],
    ["B-ORG", "B-LOC", "O"],
    ["B-PER", "O"],
    ["B-DAT", "I-DAT", "I-DAT", "O"],
]

In [63]:
from ast import mod
# 토크나이저
MODEL_NAME = 'skt/kobert-base-v1'
tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)
text = '김철수는 서울에 산다'
# 토크나이저
tokens = tokenizer.tokenize(text)
# 인코딩
encoded = tokenizer(text, return_tensors='pt') # 객체를 생성자에 넣으면.. encode
encoded

{'input_ids': tensor([[517, 490, 494,   0, 517,   0, 491,   0, 491,   0, 517,   0,   0,   0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
# NER 모델 4단계로 구성
# 1. 입력 테스트
# 2. koBERT 인코더 : 문장의 의미를 이해
# 3. 분류기(Linear) : 예측
# 4. 출력 라벨 : B-PER, O, B-LOC 이렇게 나옴

In [53]:
# 상기 내용 시현을 위한 함수 작성

import torch.nn as nn
import numpy as np
class SimpleNERModel(nn.Module):
  def __init__(self, num_labels) -> None:
    super(SimpleNERModel, self).__init__()
    self.num_labels = num_labels
    self.bert = AutoModel.from_pretrained(MODEL_NAME)   #AutoModel 출력이 없음. 그래서 마지막 은닉상태지정해주고 연결 해주는게 필요함.
    self.dropout = nn.Dropout(0.1)
    self.clf = nn.Linear(self.bert.config.hidden_size,  self.num_labels) #config.hidde_size ==> last_hidden_size에 해당
  def forward(self, input_ids, attention_mask):
    # kobert로 문자 인코딩
    outputs = self.bert(input_ids, attention_mask=attention_mask) # inputs_ids, attention_mask를 forward가 넘겨받음 
    # 마지막 은닉상태 추출 : AutoModel 출력이 없음. 그래서 마지막 은닉상태지정해주고 연결 해주는게 필요함.
    sequence_output =  outputs.last_hidden_state
    # Dropout 적용 - 과적합 방지
    sequence_output = self.dropout(sequence_output)
    # 분류기
    logits = self.clf(sequence_output)
    return logits
# 라벨의 갯수 : 왜 확인? SimpleNERMODEL에 라벨갯수를 받아야 하므로. 클래스의 갯수!  
label_list = sorted(list(set([data for i in train_labels for data in i])))
label_list
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i : label for i, label in enumerate(label_list)}
model = SimpleNERModel(num_labels=len(label_list))


# 모델 학습
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# 순전파 테스트 forward
sample_sentence = train_sentences[0]
sample_label = train_labels[0]
print(f"테스트 문장 : {' '.join(sample_sentence)}")   #sample_sentence 리스트라서 붙이기 위해 .join
print(f"정답 라벨 : {' '.join(sample_label)}")


encoding = tokenizer(sample_sentence, return_tensors='pt', truncation=True, padding=True, max_length=32, is_split_into_words=True)
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

with torch.no_grad():
  model.eval()
  logits = model(input_ids, attention_mask)
  predictions = torch.argmax(logits, dim=-1)


# 예측결과
word_ids = encoding.word_ids(batch_index=0)
pred_label = []
for i, word_idx in enumerate(word_ids):
  if word_idx is not None and i < len(predictions[0]):
    pred_label = id2label[ predictions[0][i].item() ]
    if word_idx < len(sample_sentence):
      print(f"{sample_sentence[word_idx]:10} -> {pred_label:8} 정답 : {sample_label[word_idx]}")

# 포맷코드 정리
# :<10 : 왼쪽정렬(문자열/숫자모두가능)
# :>10 : 오른쪽 정렬
# :^10 : 가운데 정렬
# :10  : 숫자형일 때만 적용 가능 → 문자열에 쓰면 오류

테스트 문장 : 김철수는 서울에 산다
정답 라벨 : B-PER B-LOC O
김철수는       -> O        정답 : B-PER
김철수는       -> O        정답 : B-PER
김철수는       -> O        정답 : B-PER
김철수는       -> O        정답 : B-PER
서울에        -> O        정답 : B-LOC
서울에        -> O        정답 : B-LOC
서울에        -> O        정답 : B-LOC
서울에        -> O        정답 : B-LOC
서울에        -> O        정답 : B-LOC
서울에        -> O        정답 : B-LOC
산다         -> O        정답 : O
산다         -> O        정답 : O


In [66]:
# 학습 DataSet
from torch.utils.data import Dataset
class NerDataSet(Dataset):
  def __init__(self,sentences,labels, tokenizer,max_len=64) -> None:
    self.sentences = sentences
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.sentences)
  def __getitem__(self,idx):
    words = self.sentences[idx]
    lbls = self.labels[idx]
    encoding = self.tokenizer(
        words,
        return_tensors='pt',
        truncation=True,
        padding=True,
        max_length=self.max_len,
        is_split_into_words=True
    )
    word_ids = encoding.word_ids(batch_index = 0)
    label_ids = []
    for w in word_ids:
      if w is None:
        label_ids.append(-100)
      else:
        label_ids.append(label2id[lbls[w]])
    return {
        'input_ids' : encoding['input_ids'].squeeze(),
        'attention_mask' : encoding['attention_mask'].squeeze(),
        'labels' : torch.tensor(label_ids)
    }

In [68]:
# 학습 DataSet
from torch.utils.data import Dataset
class NerDataSet(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len=4) -> None:
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.sentences)
    def __getitem__(self, idx):
        words = self.sentences[idx]
        ibls = self.labels[idx]
        encoding = self.tokenizer(
            words,
            return_tensors='pt',
            truncation= True,
            padding=True,
            max_length=self.max_len,
            is_split_into_words=True
        )
        word_ids = encoding.word_ids(batch_index=0)
        label_ids=[]
        for w in word_ids:
            if w is None:
                label_ids.append(-100)
            else:
                label_ids.append(label2id[ibls[w]])
        return {
            'input_ids' : encoding['input_ids'].squeeze(),
            'attention_mask' : encoding['attention_mask'].squeeze(),
            'labels' : torch.tensor(label_ids)
        }       
    

In [70]:
from torch.utils.data import DataLoader
train_dataset = NerDataSet(train_sentences, train_labels, tokenizer, max_len=4)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
next(iter(train_loader))

{'input_ids': tensor([[517, 491,   0,   0],
         [517,   0,   0,   0]]),
 'attention_mask': tensor([[1, 1, 1, 1],
         [1, 1, 1, 1]]),
 'labels': tensor([[   3,    3, -100, -100],
         [   2,    2, -100, -100]])}

In [71]:
# 모델 선언 및 학습
criterion = nn.CrossEntropyLoss(ignore_index=-100)
model=SimpleNERModel(num_labels=len(label_list))
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

In [72]:
for epoch in range(10):
    model.train()
    total_loss = 0
    for batch in train_loader:
       optimizer.zero_grad()
       input_ids = batch['input_ids'].to(device) 
       attention_mask = batch['attention_mask'].to(device)
       labels = batch['labels'].to(device)
       logits = model(input_ids, attention_mask) #skt/ ~ 모델은 input_ids, attention_mask만 받음. logit 객체는 갖고 있음
       loss = criterion(logits.view(-1, model.num_labels), labels.view(-1))
       total_loss += loss.item()
       loss.backward()
       optimizer.step()
    print(f' epoch{epoch+1} loss: {total_loss/len(train_loader):.4f}')

 epoch1 loss: 1.6340
 epoch2 loss: 1.4871
 epoch3 loss: 1.3171
 epoch4 loss: 1.2291
 epoch5 loss: 1.1002
 epoch6 loss: 1.2710
 epoch7 loss: 0.9641
 epoch8 loss: 0.9119
 epoch9 loss: 0.9528
 epoch10 loss: 0.7656


In [31]:
train_sentences[0], train_labels[0]

(['김철수는', '서울에', '산다'], ['B-PER', 'B-LOC', 'O'])

In [ ]:
# 평가
sample_sentence, sample_label =  train_sentences[0], train_labels[0]
print(sample_sentence, sample_label)
encoding = tokenizer(
        sample_sentence,
        return_tensors='pt',
        truncation=True,
        padding=True,
        max_length=20,
        is_split_into_words=True
    )
print(encoding)
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)
model.eval()
with torch.no_grad():
  logits = model(input_ids, attention_mask)
  predictions = torch.argmax(logits, dim=-1)[0]
print('결과')
word_ids = encoding.word_ids(batch_index=0)
printed_word = set()

for i ,word_idx in enumerate(word_ids):
  if word_idx is not None:
    if word_idx not in printed_word:
      printed_word.add(word_idx)
      pred_label = id2label[predictions[i].item()]
      print(f'{sample_sentence[word_idx]} -> {pred_label} 정답 : {sample_label[word_idx]}')

['김철수는', '서울에', '산다'] ['B-PER', 'B-LOC', 'O']
{'input_ids': tensor([[517, 490, 494,   0, 517,   0, 491,   0, 491,   0, 517,   0,   0,   0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
결과
김철수는 -> B-PER 정답 : B-PER
서울에 -> B-PER 정답 : B-LOC
산다 -> B-PER 정답 : O
